# Implementação no Python

## Importação de pacotes

In [110]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from time import sleep
from tqdm import tqdm # mostra a barra de carregamento
import datetime

## Funções

### `request_url`

In [65]:
def request_url(url, print_status=True):
    bad_response_cnt = 0

    headers = [
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Safari/605.1.15 (Applebot/0.1; +http://www.apple.com/go/applebot) [ip:17.241.227.148]"},
        {"User-Agent": "Mozilla/5.0 (Android 12; Mobile; rv:138.0) Gecko/138.0 Firefox/138.0 [ip:178.197.214.27]"},
        {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Safari/605.1.15 (Applebot/0.1; +http://www.apple.com/go/applebot) [ip:17.241.227.148]"},
        {"User-Agent": "TuneIn Radio Pro/31.9.0; iPhone17,1; iOS/18.5"}
    ]

    user_message = {
    200: 'URL requisitada com sucesso.',
    201: 'Recurso criado com sucesso.',
    202: 'Requisição aceita para processamento.',
    204: 'Requisição bem-sucedida, sem conteúdo retornado.',
    301: 'Recurso movido permanentemente.',
    302: 'Recurso movido temporariamente.',
    400: 'Requisição inválida.',
    401: 'Não autorizado. Autenticação necessária.',
    403: 'Proibido. Você não tem permissão para acessar este recurso.',
    404: 'Recurso não encontrado.',
    405: 'Método HTTP não permitido para o recurso.',
    408: 'Tempo de requisição esgotado.',
    409: 'Conflito. O recurso já existe ou há um problema de estado.',
    500: 'Erro interno do servidor.',
    501: 'Funcionalidade não implementada no servidor.',
    502: 'Bad Gateway. Erro de comunicação entre servidores.',
    503: 'Serviço indisponível. Tente novamente mais tarde.',
    504: 'Gateway Timeout. Tempo limite ao esperar resposta do servidor.'
                    }
    
    while bad_response_cnt < len(headers)-1:
        response = requests.get(url, headers=headers[bad_response_cnt])
        requisition_status = response.status_code//100
        
        match requisition_status:
            case 2 | 3:
                if print_status:
                    print(user_message[response.status_code])
                return response
                
            case 4 | 5:
                bad_response_cnt += 1
                sleep(0.5)

    print(user_message[response.status_code])
    return response

### `save_data`

In [76]:
def save_data(data:dict):
    df = pd.DataFrame(data)
    df.to_csv('result.csv', index=False, encoding='utf-8-sig')
    print('Dados salvos com sucesso.')

### `save_df`

In [121]:
def save_df(df):
    df.to_csv('result.csv', index=False, encoding='utf-8-sig')
    print('Dados salvos com sucesso.')

### `scrape_uol()`

In [ ]:
def scrape_uol():
    """ 
    Função que tem como objetivo coletar do xml do sitemap do uol todas as urls e datas de modificação das notícias, bem
    como o título das manchetes ao acessar cada url individualmente.

    Observações: Na página do uol, no head do html estão inclusas as meta tags, o título encontra-se nessa '# <meta property="og:title"'
                 Exemplo: # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
                            
    """
    
    target_url = 'https://noticias.uol.com.br/sitemap/v2/today.xml'
    response = request_url(target_url)  # coletando o xml do uol
    
    soup = bs(response.content, "xml")

    # monto o dicionário contendo as chaves
    urls_data = [
        {
            "Veículo": 'UOL'
            "Link da matéria": url.loc.text,
            "Título da matéria": "",
            "Subtítulo": "",
            "Data (em ISO)": url.lastmod.text
        }
        for url in soup.find_all("url")]

    max_error_cnt = 3

    print(f'Processando urls...')
    for i in tqdm(range(len(urls_data))):
            error_cnt = 0
    
            while error_cnt < max_error_cnt:
                try:
                    response = request_url(urls_data[i]['url'], print_status=False)
                    
                    if response:
                            soup = bs(response.content, "html.parser")
                            title = soup.select('meta[property="og:title"]')[0]['content']  # o resultado retornado é uma lista com 1 elemento
                            urls_data[i]['title'] = title
                            break
                except Exception as e:
                    error_cnt += 1

### `collect_recent_news`

In [119]:
def collect_recent_news(data: dict, n=10):
    df = pd.DataFrame(data)
    df['Data (em ISO)'] = pd.to_datetime(df['Data (em ISO)'])

    df.sort_values('Data (em ISO)', ascending=False)

    return df[0:n]

## Fazendo uma requisição

#### User agents

Ao tentar fazer uma série de requisições, recebi o código 403, ou seja, o servidor está bloqueando meu acesso. Uma razão para isso pode ser o fato de não ter sido definido um user-agent logo no início. Ou seja, um cabeçalho enviado junto com a requisição que faz com que ela soe "mais humana". Site para coletar user-agents: https://user-agents.net/

In [66]:
urls = ['https://g1.globo.com/rss/g1/educacao/',
        'https://noticias.uol.com.br/sitemap/v2/today.xml',
        'https://www.r7.com/arc/outboundfeeds/sitemap-news/?outputType=xml']

for url in urls:
    response = request_url(url)

URL requisitada com sucesso.
URL requisitada com sucesso.
URL requisitada com sucesso.


## Raspando dados - `noticias.uol.com.br/sitemap/v2/today.xml`

Coletando dados das tags \<url>

In [42]:
target_url = 'https://noticias.uol.com.br/sitemap/v2/today.xml'
response = request_url(target_url)  # coletando o xml do uol

soup = bs(response.content, "xml")

urls_data = [
    {
        "url": url.loc.text,
        "title": "",
        "subtitle": "",
        "ISO_datetime": url.lastmod.text
    }
    for url in soup.find_all("url")]

urls_data[0:2]

https://noticias.uol.com.br/sitemap/v2/today.xml status - ok


[{'url': 'https://noticias.uol.com.br/politica/ultimas-noticias/2025/06/27/moraes-nega-anular-delacao-mauro-cid.htm',
  'title': '',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:58:25.000Z'},
 {'url': 'https://noticias.uol.com.br/cotidiano/ultimas-noticias/2025/06/27/rios-em-sao-paulo.htm',
  'title': '',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:57:32.000Z'}]

##### Teste - Fazendo uma chave para cada url individualmente e coletar o título

In [43]:
response = verify_status_code(urls_data[0]['url'])
if response:
        soup = bs(response.content, "html.parser")
        # no head estão inclusas as meta tags, o título encontra-se nessa
        # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
        title = soup.select('meta[property="og:title"]')[0]['content']  # o resultado retornado é uma lista com 1 elemento

https://noticias.uol.com.br/politica/ultimas-noticias/2025/06/27/moraes-nega-anular-delacao-mauro-cid.htm status - ok


In [67]:
max_error_cnt = 3

print(f'Processando urls...')
for i in tqdm(range(len(urls_data))):
        error_cnt = 0

        while error_cnt < max_error_cnt:
            try:
                response = request_url(urls_data[i]['url'], print_status=False)
                
                if response:
                        soup = bs(response.content, "html.parser")
                        # no head estão inclusas as meta tags, o título encontra-se nessa
                        # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
                        title = soup.select('meta[property="og:title"]')[0]['content']  # o resultado retornado é uma lista com 1 elemento
                        urls_data[i]['title'] = title
                        break
            except Exception as e:
                error_cnt += 1
    

urls_data

Processando urls...


100%|████████████████████████████████████████████████████████████████████████████████| 598/598 [04:08<00:00,  2.41it/s]


[{'url': 'https://noticias.uol.com.br/politica/ultimas-noticias/2025/06/27/moraes-nega-anular-delacao-mauro-cid.htm',
  'title': 'Moraes nega pedido de defesa de Câmara para anular delação de Mauro Cid',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:58:25.000Z'},
 {'url': 'https://noticias.uol.com.br/cotidiano/ultimas-noticias/2025/06/27/rios-em-sao-paulo.htm',
  'title': 'Será que na sua tem? Há mais de 200 rios embaixo das ruas de SP; veja mapa',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:57:32.000Z'},
 {'url': 'https://noticias.uol.com.br/ultimas-noticias/reuters/2025/06/27/ruanda-e-congo-assinam-acordo-de-paz-nos-eua-para-por-fim-a-conflitos-e-atrair-investimentos.htm',
  'title': 'Ruanda e Congo assinam acordo de paz nos EUA para pôr fim a conflitos e atrair investimentos',
  'subtitle': '',
  'ISO_datetime': '2025-06-27T21:54:46.000Z'},
 {'url': 'https://noticias.uol.com.br/cotidiano/ultimas-noticias/2025/06/27/brf-deve-indenizar-em-r-150-mil-funcionaria-que-perdeu-g

### Resultado final

In [153]:
def scrape_uol(n=10):
    """ 
    Função que tem como objetivo coletar do xml do sitemap do uol todas as urls e datas de modificação das notícias, bem
    como o título das manchetes ao acessar cada url individualmente.

    Observações: Na página do uol, no head do html estão inclusas as meta tags, o título encontra-se nessa '# <meta property="og:title"'
                 Exemplo: # <meta property="og:title" content="Carla Araújo: Motta surpreende com reviravolta política para queda do IOF">
                            
    """
    
    target_url = 'https://noticias.uol.com.br/sitemap/v2/today.xml'
    response = request_url(target_url)  # coletando o xml do uol
    
    soup = bs(response.content, "xml")

    # monto o dicionário contendo as chaves
    news_dict = [
        {
            "Veículo": 'UOL',
            "Link da matéria": url.loc.text,
            "Título da matéria": "",
            "Subtítulo": "",
            "Data (em ISO)": url.lastmod.text
        }
        for url in soup.find_all("url")]

    news_df = collect_recent_news(news_dict, n)
    
    max_error_cnt = 3
    print(f'Processando urls - UOL')
    for i in range(len(news_df)):
            error_cnt = 0
    
            while error_cnt < max_error_cnt:
                try:
                    response = request_url(news_df.loc['Link da matéria'][i], print_status=False)
                    
                    if response:
                            soup = bs(response.content, "html.parser")
                            title = soup.select('meta[property="og:title"]')[0]['content']  # o resultado retornado é uma lista com 1 elemento
                            news_df.loc[i, "Título da matéria"] = title
                            break
                except Exception as e:
                    error_cnt += 1
                

    print('Urls processadas com suesso.')
    return news_df

## Raspando dados -`https://g1.globo.com/rss/g1/educacao/`

## Salvando dados no excel

In [91]:
def save_data(data:dict):
    df = pd.DataFrame(data)
    df.to_csv('result.csv', index=False, encoding='utf-8-sig')
    print('Dados salvos com sucesso.')

## Pipeline

In [150]:
df.loc[2,"Título da matéria"]

'Resultado da Quina de São João, concurso 6760: veja dezenas sorteadas'

In [154]:
df = scrape_uol()

URL requisitada com sucesso.
Processando urls - UOL


In [127]:
data

[{'Veículo': 'UOL',
  'Link da matéria': 'https://noticias.uol.com.br/ultimas-noticias/agencia-brasil/2025/06/28/defesa-civil-no-sul-pede-a-desalojados-que-nao-voltem-as-suas-casas.htm',
  'Título da matéria': 'Defesa Civil no Sul pede a desalojados que não voltem às suas casas ',
  'Subtítulo': '',
  'Data (em ISO)': '2025-06-28T18:04:21.000Z'},
 {'Veículo': 'UOL',
  'Link da matéria': 'https://noticias.uol.com.br/ultimas-noticias/agencia-brasil/2025/06/28/roberta-nistra-segue-linhagem-da-mulher-sambista-que-toca-cavaquinho.htm',
  'Título da matéria': 'Roberta Nistra segue linhagem da mulher sambista que toca cavaquinho',
  'Subtítulo': '',
  'Data (em ISO)': '2025-06-28T18:04:21.000Z'},
 {'Veículo': 'UOL',
  'Link da matéria': 'https://noticias.uol.com.br/cotidiano/ultimas-noticias/2025/06/28/homem-invade-casa-de-ex-e-mata-mulher-com-socos-e-chutes-no-litoral-de-sp.htm',
  'Título da matéria': 'Homem invade casa de ex e mata mulher com socos e chutes no litoral de SP',
  'Subtítulo'

In [123]:
df = collect_recent_news(data)
# df

,Veículo,Link da matéria,Título da matéria,Subtítulo,Data (em ISO)
0,UOL,https://noticias.uol.com.br/ultimas-noticias/a...,Defesa Civil no Sul pede a desalojados que não...,,2025-06-28 18:04:21+00:00
1,UOL,https://noticias.uol.com.br/ultimas-noticias/a...,Roberta Nistra segue linhagem da mulher sambis...,,2025-06-28 18:04:21+00:00
2,UOL,https://noticias.uol.com.br/cotidiano/ultimas-...,Homem invade casa de ex e mata mulher com soco...,,2025-06-28 18:02:44+00:00
3,UOL,https://noticias.uol.com.br/ultimas-noticias/d...,Multidão vai à parada LGBTQ+ de Budapeste em d...,,2025-06-28 18:02:36+00:00
4,UOL,https://noticias.uol.com.br/newsletters/novos-...,Caso Daniel Silveira escancara hipocrisia bols...,,2025-06-28 18:00:48+00:00
5,UOL,https://noticias.uol.com.br/colunas/leonardo-s...,Leonardo Sakamoto: Caso Daniel Silveira escanc...,,2025-06-28 18:00:43+00:00
6,UOL,https://noticias.uol.com.br/cotidiano/ultimas-...,Resultado da Quina de São João 2025: que horas...,,2025-06-28 18:00:00+00:00
7,UOL,https://noticias.uol.com.br/ultimas-noticias/a...,Milhares participam da Marcha do Orgulho em Bu...,,2025-06-28 17:59:48+00:00
8,UOL,https://noticias.uol.com.br/ultimas-noticias/a...,Onda de calor se espalha pelo sul da Europa,,2025-06-28 17:54:44+00:00
9,UOL,https://noticias.uol.com.br/ultimas-noticias/d...,Como os alemães financiam indiretamente um núc...,,2025-06-28 17:39:35+00:00


In [125]:
save_df(df)

Dados salvos com sucesso.


## Análise de dados

In [106]:
df = pd.DataFrame(data)
df.head()

df[df['Data (em ISO)'] ==""].sum()

Veículo              0
Link da matéria      0
Título da matéria    0
Subtítulo            0
Data (em ISO)        0
dtype: object

In [109]:
(df['Subtítulo'] =="").sum()

np.int64(464)

## Testes

In [116]:
iso_str = '2025-06-28T18:10:48.000Z'
data = pd.to_datetime(iso_str)

Timestamp('2025-06-28 18:10:48+0000', tz='UTC')